# Сравнение open-source солверов на примере задачи ритейла

Привет, Habr! На связи отдел аналитики данных X5 Tech.

В этой статье мы продолжим говорить про оптимизаторы.
Для каких задач и как они могут применяться в python мы поговорили в [предыдущей статье](http://).

Здесь мы сделаем небольшой обзор существующих open-source решений в python, затронем их различия, особенности и какие задачи могут решать.


## Обзор пакетов python


Рассмотрим open-source пакеты и солверы для решения задач условной оптимизации в зависимости от ее типа, которые часто можно встретить в практических задачах.

### Открытые библиотеки, предоставляющие интерфейс для решения оптимизационных задач


[__Scipy__](https://scipy.org/) - библиотека, которая содержит большой набор функций для научных вычислений,
в том числе имеет инструменты для решения оптимизационных задач, находящиеся в модуле __scipy.optimize__.
В модуле находятся методы для решения задач нелинейного программирования (__NLP__), линейного программирования (__LP__) и смешанного целочисленного линейного програмимрования(__MILP__).


Среди солверов, которые поддерживают решение задач условной оптимизации(NLP) можно выделить ___cobyla___, ___slsqp___, ___trust-constr___. Описание методов и ссылки на статьи можно найти [тут](https://habr.com/ru/company/ods/blog/448054/). Здесь вкратце отметим, что __сobyla__ - это метод, позволяющий производить оптимизацию функции, градиент которой неизвестен, т.е. по сути заниматься оптимизацией "черного ящика", также cobyla не поддерживает ограничения типа равенства и границы для переменных $x$ - их необходимо задавать через ограничения. Что каксается __slsqp__, __trust-constr__, то для увеличения устойчивости и сходимости оптимизаторов, функции(целевую и ограничения) необходимо отмасштабировать так, чтобы принимаемые значения имели порядок единицы.

Рассмотрим небольшой пример примениния метода minimize для решения задачи условной оптимизации, основанный на постановке задачи ценообразования из статьи 1, заодно рассмотрим как ведет себя решение, когда целевая функция имеет "большой" масштаб.

In [9]:
# мини пример из постановки первой статьи
import scipy.optimize as scopt
import numpy as np

# задаем эластичности, используемые в формуле Q = Q0 * exp(E * (P/P0 - 1))
E = np.array([-3., -1., -0.5])
# текущие цены
P0 = np.array([10., 10., 10.])
# текущие продажи
Q0 = np.array([500000., 2000000., 300000.0])
# себестоимость
C = np.array([9.0, 8.0, 7.0])
# текущая выручка
R0 = np.sum(P0 * Q0)
# текущая маржа
M0 = np.sum((P0 - C) * Q0)

# выручка - целевая функция, задаем возможность "управлять" масштабом через 'scale'
def f_obj(x, args):
    f = - args['scale'] * np.sum(Q0 * P0 * x * np.exp(E * (x-1.)))
    return f
obj = f_obj

# функция для ограничения по марже, по умолчанию отмасштабиируем ограничения на текущую выручку
def f_cons(x):
    f = np.sum(Q0 * (P0 * x - C) * np.exp(E * (x-1.0))) / R0
    return f
cons = [scopt.NonlinearConstraint(f_cons, lb=M0 / R0, ub=np.inf)]

# поиск новой цены производим в диапазоне 90% - 110% от текущей цены
x_bounds = [(0.9, 1.1)] * 3
# стартовая точка для поиска
x0 = [1.0] * 3

res_nonscaled = scopt.minimize(obj, x0, bounds=x_bounds, constraints=cons, method='slsqp', args={'scale': 1.})
res_scaled = scopt.minimize(obj, x0, bounds=x_bounds, constraints=cons, method='slsqp', args={'scale': 1.0 / R0})
print('Решение с масштабированием ', np.round(res_scaled['x'], 3), res_scaled['message'])
print('Решение без масштабирования',np.round(res_nonscaled['x'], 3), res_nonscaled['message'])

Решение с масштабированием  [0.9   1.016 1.1  ] Optimization terminated successfully
Решение без масштабирования [1. 1. 1.] Optimization terminated successfully


Как мы видим, в случае когда целевая функция не отмасштабирована солвер даже не трогается с места - решение это стартовое значение, более того согласно своим внутренним критериям считает, что решение является оптимальным, хотя на деле это не так - этот необходимо иметь это ввиду при реализации.

Для решения задач линейного программирования в подмодуле имеется функция [linprog](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html#scipy.optimize.linprog), начиная с версии scipy==1.9.0 появилась возможность решения задачи линейного целочисленного программирования с помощью функции [milp](https://scipy.github.io/devdocs/reference/generated/scipy.optimize.milp.html) и linprog. В качестве солвера LP(MILP) по умолчанию использется [HiGHS](https://www.maths.ed.ac.uk/hall/HiGHS/)


In [10]:
# пример с "рюкзаком" у которого типы переменных различаются
c = -np.array([1, 2, 3])
A_ub = np.array([[2, 1, 3]])
b_ub = np.array([7])
var_types = [1, 1, 1]
bounds = [(0, 1), (0, 2), (0, 1)]
res_milp = scopt.linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=var_types, method='highs')
res_milp

           con: array([], dtype=float64)
 crossover_nit: -1
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -8.0
       ineqlin:  marginals: array([0.])
  residual: array([0.])
         lower:  marginals: array([0., 0., 0.])
  residual: array([1., 2., 1.])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: -1
         slack: array([0.])
        status: 0
       success: True
         upper:  marginals: array([0., 0., 0.])
  residual: array([0., 0., 0.])
             x: array([1., 2., 1.])

[__Pyomo__](http://www.pyomo.org/) - пакет, который содержит ряд инструментов для формулирования, решения и анализа оптимизационных моделей.
Главная особенность — это удобный интерфейс для структурированного формулирования оптимизационной задачи и поддержка большого количества солверов, в том числе коммерческих.
Pyomo преобразует сформулированную модель в формат, понятный для запускаемого солвера.

Pyomo входит в проект [COIN-OR](https://www.coin-or.org/), содержащий ряд солверов, среди которых выделим два:

1) [___Ipopt___](https://github.com/coin-or/Ipopt) - находит локальные оптимумы в задаче NLP с помощью прямо-двойственного метода внутренней точки, подробнее в оригинальной [статье](http://www.optimization-online.org/DB_HTML/2004/03/836.html).
2) [___Cbc___](https://github.com/coin-or/Cbc) - решает задачи MILP, на базе алгоритма, сочетающем в себе метод ветвей и границ и секущих плоскостей [wiki](https://en.wikipedia.org/wiki/Branch_and_cut).

Также для решения задач LP(MILP) имеется поддержка пакета [glpk](https://en.wikipedia.org/wiki/GNU_Linear_Programming_Kit).
Отметим еще один солвер [bonmin](https://github.com/coin-or/Bonmin), который построен поверх __Cbc__ и __Ipopt__ - сочетание двух солверов, которое браться за задачи __MINLP__.


In [111]:
# пример аналогичный предыдущему
# мини пример из постановки первой статьи
import pyomo.environ as pyo
import numpy as np

# Количество товаров
N = 3
# задаем эластичности, используемые в формуле Q = Q0 * exp(E * (P/P0 - 1))
E = np.array([-3., -1., -0.5])
# текущие цены
P0 = np.array([10., 10., 10.])
# текущие продажи
Q0 = np.array([500000., 2000000., 300000.0])
# себестоимость
C = np.array([9.0, 8.0, 7.0])
# текущая выручка
R0 = np.sum(P0 * Q0)
# текущая маржа
M0 = np.sum((P0 - C) * Q0)

bounds = [(0.9, 1.1)] * N

model = pyo.ConcreteModel('model')

model.x = pyo.Var(range(N), domain=pyo.Reals, bounds=bounds, initialize=1)
scale = 1.0

obj_expr = sum(scale * P0[i] * model.x[i] * Q0[i] * pyo.exp(E[i] * (model.x[i] - 1)) for i in model.x)
model.obj = pyo.Objective(expr=obj_expr, sense=pyo.maximize)

con_expr = sum((P0[i] * model.x[i] - C[i]) * Q0[i] * pyo.exp(E[i] * (model.x[i] - 1)) for i in model.x) >= M0
model.con = pyo.Constraint(expr=con_expr)

solver = pyo.SolverFactory('ipopt')
res = solver.solve(model)
x_opt = [round(model.x[i].value, 3) for i in model.x]
print(x_opt, '; obj value = ', round(model.obj(x_opt), 0))

[0.9, 1.016, 1.1] ; obj value =  29210742.0


In [113]:
# milp примерчик с рюкзаком
# пример с "рюкзаком" у которого типы переменных различаются
c = np.array([1, 2, 3])
A = np.array([2, 1, 3])
b = 7

model = pyo.ConcreteModel('model')
bounds = [(0, 1), (0, 2), (0, 1)]

model.x = pyo.Var(range(3), domain=pyo.Integers, bounds=bounds)

obj_expr = sum(c * model.x)
model.obj = pyo.Objective(expr=obj_expr, sense=pyo.maximize)

con_expr = sum(A * model.x) <= b
model.con = pyo.Constraint(expr=con_expr)

solver = pyo.SolverFactory('glpk')
res = solver.solve(model)
x_opt = [model.x[i].value for i in model.x]
print(x_opt, '; obj value = ', model.obj(x_opt))

[1.0, 2.0, 1.0] ; obj value =  8.0


Pyomo имеет подмодуль __GDP__ (Generalized Disjunctive Programming) - который позволяет моделировать логические правила и задавать ограничения, которые должны при этом выполняться, в простейшем случае данный подход может быть удобне когда необходимо выбрать одно из действий, каждое из которых описывается своей системой ограничений.

In [112]:
import pyomo.environ as pyo
from pyomo.gdp import Disjunct, Disjunction
model = pyo.ConcreteModel('gdp_sample')
model.x = pyo.Var(range(0, 3), domain=pyo.Reals, bounds=(-3., 3.))
a = [0.0, 1.0, -2.0]
obj_expr = sum((model.x[i] - a[i]) ** 2 for i in model.x)
model.obj = pyo.Objective(expr=obj_expr, sense=pyo.minimize)
model.djn = Disjunction(range(3))

d = Disjunct()
d.c = pyo.Constraint(rule=(0, model.x[0], 1))
for i in range(3):
    model.djn[i] = [model.x[i] <= -1.5, model.x[i] == 0, model.x[i] >= 1.5] 

    
pyo.TransformationFactory('gdp.hull').apply_to(model)
solver = pyo.SolverFactory('bonmin')
res = solver.solve(model)
x_opt = [round(model.x[i].value, 3) for i in model.x]
x_opt


[0.0, 1.5, -2.0]

[__Cvxpy__](https://www.cvxpy.org/index.html) - данный пакет специально заточен для решения задач выпуклой оптимизации (convex optimization).
После того как задача сформулирована, перед решением проверяется выпуклость и аффинность целевой функции и ограничений с помощью правил [DCP](https://www.cvxpy.org/tutorial/dcp/index.html)
(disciplined convex programming). По сути это набор правил, которые однозначно гарантируют выпуклость функции, например, если берется
После проверки задача преобразуется в стандартную форму и передается квадратичному или коническому солверу.
Полный список солверов и, соответственно, тиопв решаемых задач можно найти [здесь](https://www.cvxpy.org/tutorial/advanced/index.html).


In [40]:
import cvxpy as cp

# пример, когда не выполняются правила DCP, но при этом функция выпуклая
X1, Y1 = 0.0, 0.0
X2, Y2 = 1.0, 2.0
# N = V2 / V1
N = 1.5

X = cp.Variable(1)
Y_ = 1.0

objective = cp.norm(cp.hstack([X-X1, Y_-Y1]), 2) + N * cp.norm(cp.hstack([X2-X, Y2-Y_]), 2)
constraints = []
constraints.extend([X >= 0.0, X <= X2])
problem = cp.Problem(cp.Minimize(objective), constraints)
obj_val = problem.solve('ECOS')
X_ = X.value[0]
print(f'x_opt = {round(X_, 3)}')

x_opt = 0.623


In [38]:
objective = cp.sqrt(cp.square(X - X1) + cp.square(Y_ - Y1) ** 2) + cp.sqrt(cp.square(X2 - X) + cp.square(Y2 - Y_) ** 2)
constraints = []
constraints.extend([X >= 0.0, X <= X2])
problem = cp.Problem(cp.Minimize(objective), constraints)
problem.is_dcp(), problem.is_dgp(), problem.is_dpp(), problem.is_dqcp(), problem.is_qp()

(False, False, False, False, False)

In [ ]:
import cvxpy as cp

x = cp.Variable(3, boolean=True)
c = np.array([1., 2., 3.])
A = np.array([2., 1., 3.])
b = 5.
obj = cp.Maximize(cp.sum(c @ x))
cons = [(A @ x) <= b]
prb = cp.Problem(obj, cons)
sol = prb.solve(verbose=False, solver='GLPK_MI')

Составим таблицу для перечисленных пакетов и солверов, какие типы задач они решают:


|  Пакеты в python  |     Солвер(метод)    | NLP | LP | MILP | MINLP |
|-------------------|----------------------|-----|----|------|-------|
| scipy             | cobyla               | y   | n  | n    | n     |
| scipy             | slsqp                | y   | n  | n    | n     |
| scipy             | trust-constr         | y   | n  | n    | n     |
| pyomo             | ipopt                | y   | y  | n    | n     |
| pyomo, cvxpy      | glpk                 | n   | y  | y    | n     |
| pyomo, cvxpy      | cbc                  | n   | y  | y    | n     |
| cvxpy             | ecos                 | y   | y  | y    | n     |
| pyomo             | bonmin               | y   | y  | y    | y     |



## Заключение

В данной статье мы рассмотрели ряд open-source библиотеки для решения оптимизационых задач